In [1]:
import pandas as pd
from pathlib import Path

# Get project root (go up from scripts/bronze_to_silver to project root)
PROJECT_ROOT = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
file_path = PROJECT_ROOT / "data" / "silver" / "bronze_outliers_handled.csv"

df = pd.read_csv(file_path)
df.head()

,subject_id,hadm_id,admittime,dischtime,gender,anchor_age,admission_type,admission_location,discharge_location,race,...,resp_rate,o2sat,BP_systolic,BP_diastolic,other_uti,other_uti_present,has_cauti_history,pain_documented,ventilator_used,y
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,F,52,URGENT,TRANSFER FROM HOSPITAL,HOME,WHITE,...,16.0,98.0,110.0,65.0,[],False,False,True,False,0
1,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,F,52,EW EMER.,EMERGENCY ROOM,HOME,WHITE,...,24.0,92.0,98.0,66.0,[],False,False,True,True,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,F,52,EW EMER.,EMERGENCY ROOM,HOSPICE,WHITE,...,18.0,99.0,98.0,66.0,[],False,False,True,True,0
3,10000560,28979390,2189-10-15 10:30:00,2189-10-17 15:00:00,F,53,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,WHITE,...,NaN,NaN,104.0,68.0,[],False,False,True,False,0
4,10000690,25860671,2150-11-02 18:02:00,2150-11-12 13:45:00,F,86,EW EMER.,EMERGENCY ROOM,REHAB,WHITE,...,35.0,83.0,130.0,68.0,[],False,False,False,False,0


In [2]:
missing_summary = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percent": (df.isna().mean() * 100).round(2)
}).sort_values("missing_percent", ascending=False)

# Add non-missing count
missing_summary["non_missing_count"] = len(df) - missing_summary["missing_count"]

# Create combined column in the format "Missing - num1; Non-Missing - num2"
missing_summary["missing_non_missing"] = (
    "Missing - " + missing_summary["missing_count"].astype(str) + 
    "; Non-Missing - " + missing_summary["non_missing_count"].astype(str)
)

missing_summary

# Save missing summary to CSV
output_path = PROJECT_ROOT / "data" / "silver" / "missing_summary.csv"
missing_summary.to_csv(output_path, index=True)
print(f"Missing summary saved to: {output_path}")


Missing summary saved to: C:\Users\Coditas\Desktop\Projects\Cauti\data\silver\missing_summary.csv


In [3]:
missing_only = df.loc[:, df.isna().any()]
print(missing_only.columns.tolist())

['discharge_location', 'final_insertion_date', 'final_removal_date', 'catheter_duration_days', 'BMI', 'catheter_size', 'urinalysis_wbc', 'urinalysis_rbc', 'urinalysis_nitrite', 'urine_bacteria', 'blood_wbc', 'creatinine', 'blood_crp', 'urine_output_ml', 'cfu_count', 'temperature', 'heart_rate', 'resp_rate', 'o2sat', 'BP_systolic', 'BP_diastolic']


In [4]:
missing_only = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percentage": (df.isna().mean() * 100).round(2)
})

# Keep only columns that actually have missing values
missing_only = missing_only[missing_only["missing_count"] > 0]

# Sort by highest missing percentage
missing_only = missing_only.sort_values("missing_percentage", ascending=False)

print(missing_only)


                        missing_count  missing_percentage
urine_bacteria                 157797               99.86
catheter_duration_days         156447               99.00
final_removal_date             156260               98.89
blood_crp                      147072               93.07
cfu_count                      145635               92.16
urine_output_ml                137052               86.73
final_insertion_date           130051               82.30
urinalysis_wbc                 110558               69.96
urinalysis_rbc                 109895               69.54
catheter_size                  105328               66.65
urinalysis_nitrite              89116               56.40
BP_diastolic                    55647               35.22
BP_systolic                     55651               35.22
temperature                     47963               30.35
o2sat                           47659               30.16
resp_rate                       47067               29.79
heart_rate    

## BMI Handling Missing Values

In [6]:
# Sort by target + patient + admission time
df = df.sort_values(["y", "subject_id", "admittime"])

# --------------------------------------------------
#  Forward fill BMI within (y, subject_id)
# --------------------------------------------------
df["BMI"] = df.groupby(["y", "subject_id"])["BMI"].ffill()

# --------------------------------------------------
#  Backward fill BMI within (y, subject_id)
# --------------------------------------------------
df["BMI"] = df.groupby(["y", "subject_id"])["BMI"].bfill()

# --------------------------------------------------
# Missing summary (after ffill/bfill)
# --------------------------------------------------
null_count = df["BMI"].isna().sum()
total = len(df)
percent_null = round((null_count / total) * 100, 2)

print("BMI Null Count:", null_count)
print("Total Rows:", total)
print("Percentage Null:", percent_null, "%")


# --------------------------------------------------
#  Create age bins (same as before)
# --------------------------------------------------
bins = [0, 30, 45, 60, 75, 120]
labels = ["<30", "30-45", "45-60", "60-75", "75+"]

df["age_group"] = pd.cut(df["anchor_age"], bins=bins, labels=labels, right=False)


# --------------------------------------------------
#  Compute median BMI per (y, age_group, gender)
# --------------------------------------------------
group_medians = (
    df.groupby(["y", "age_group", "gender"], observed=False)["BMI"]
      .median()
)


# --------------------------------------------------
#  Fill remaining nulls using class-specific medians
# --------------------------------------------------
def fill_bmi(row):
    if pd.isna(row["BMI"]):
        return group_medians.loc[row["y"], row["age_group"], row["gender"]]
    return row["BMI"]

df["BMI"] = df.apply(fill_bmi, axis=1)


# --------------------------------------------------
#  Cleanup
# --------------------------------------------------
df.drop(columns=["age_group"], inplace=True)


# --------------------------------------------------
# Final missing summary
# --------------------------------------------------
null_count = df["BMI"].isna().sum()
total = len(df)
percent_null = round((null_count / total) * 100, 2)

print("BMI Null Count:", null_count)
print("Total Rows:", total)
print("Percentage Null:", percent_null, "%")


BMI Null Count: 44298
Total Rows: 158020
Percentage Null: 28.03 %
BMI Null Count: 0
Total Rows: 158020
Percentage Null: 0.0 %


## urinalysis_wbc Missing values handling 

In [8]:
import pandas as pd
import numpy as np

# make sure age_group exists
df["age_group"] = pd.cut(
    df["anchor_age"],
    bins=[0, 18, 40, 65, 120],
    labels=["child", "adult", "middle_age", "elderly"]
)

# --------------------------------------------------
# 1️⃣ group median by (y, age_group, gender)
# --------------------------------------------------
group_median = (
    df.groupby(["y", "age_group", "gender"], observed=False)["urinalysis_wbc"]
      .median()
)

# --------------------------------------------------
# 2️⃣ join group_median back onto df
# --------------------------------------------------
df = df.join(
    group_median.rename("urinalysis_wbc_group_median"),
    on=["y", "age_group", "gender"]
)

# --------------------------------------------------
# 3️⃣ fallback overall median *within each y*
# --------------------------------------------------
overall_median_by_y = df.groupby("y")["urinalysis_wbc"].median()

df["urinalysis_wbc_group_median"] = df.apply(
    lambda row: (
        overall_median_by_y.loc[row["y"]]
        if pd.isna(row["urinalysis_wbc_group_median"])
        else row["urinalysis_wbc_group_median"]
    ),
    axis=1
)

# --------------------------------------------------
# 4️⃣ fill missing urinalysis_wbc
# --------------------------------------------------
df["urinalysis_wbc"] = df["urinalysis_wbc"].fillna(
    df["urinalysis_wbc_group_median"]
)

# --------------------------------------------------
# 5️⃣ cleanup
# --------------------------------------------------
df = df.drop(columns=["urinalysis_wbc_group_median"])


# --------------------------------------------------
# Missing summary
# --------------------------------------------------
null_count = df["urinalysis_wbc"].isna().sum()
total = len(df)
percent_null = round((null_count / total) * 100, 2)

print("urinalysis_wbc Null Count:", null_count)
print("Total Rows:", total)
print("Percentage Null:", percent_null, "%")


urinalysis_wbc Null Count: 0
Total Rows: 158020
Percentage Null: 0.0 %


## urinalysis_rbc Missing values handling

In [10]:
import pandas as pd
import numpy as np

# Ensure age_group exists (safe to repeat)
df["age_group"] = pd.cut(
    df["anchor_age"],
    bins=[0, 18, 40, 65, 120],
    labels=["child", "adult", "middle_age", "elderly"]
)

# --------------------------------------------------
# Group median by (y, age_group, gender)
# --------------------------------------------------
group_median_rbc = (
    df.groupby(["y", "age_group", "gender"], observed=False)["urinalysis_rbc"]
      .median()
)

# --------------------------------------------------
# Join onto df
# --------------------------------------------------
df = df.join(
    group_median_rbc.rename("urinalysis_rbc_group_median"),
    on=["y", "age_group", "gender"]
)

# --------------------------------------------------
# Fallback median within each y
# --------------------------------------------------
overall_median_rbc_by_y = df.groupby("y")["urinalysis_rbc"].median()

df["urinalysis_rbc_group_median"] = df.apply(
    lambda row: (
        overall_median_rbc_by_y.loc[row["y"]]
        if pd.isna(row["urinalysis_rbc_group_median"])
        else row["urinalysis_rbc_group_median"]
    ),
    axis=1
)

# --------------------------------------------------
#  Fill missing values
# --------------------------------------------------
df["urinalysis_rbc"] = df["urinalysis_rbc"].fillna(
    df["urinalysis_rbc_group_median"]
)

# --------------------------------------------------
#  Cleanup
# --------------------------------------------------
df = df.drop(columns=["urinalysis_rbc_group_median"])

# --------------------------------------------------
# Missing summary
# --------------------------------------------------
null_count = df["urinalysis_rbc"].isna().sum()
total = len(df)
percent_null = round((null_count / total) * 100, 2)

print("urinalysis_rbc Null Count:", null_count)
print("Total Rows:", total)
print("Percentage Null:", percent_null, "%")


urinalysis_rbc Null Count: 0
Total Rows: 158020
Percentage Null: 0.0 %


## blood_wbc Missing values handling

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [12]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# # Ensure age_group exists
# df["age_group"] = pd.cut(
#     df["anchor_age"],
#     bins=[0, 18, 40, 65, 120],
#     labels=["child", "adult", "middle_age", "elderly"]
# )

# # --------------------------------------------------
# #  Group median by (y, age_group, gender)
# # --------------------------------------------------
# group_median_blood_wbc = (
#     df.groupby(["y", "age_group", "gender"], observed=False)["blood_wbc"]
#       .median()
# )

# # --------------------------------------------------
# #  Join group median onto df
# # --------------------------------------------------
# df = df.join(
#     group_median_blood_wbc.rename("blood_wbc_group_median"),
#     on=["y", "age_group", "gender"]
# )

# # --------------------------------------------------
# #  Fallback median within each y
# # --------------------------------------------------
# overall_median_blood_wbc_by_y = df.groupby("y")["blood_wbc"].median()

# df["blood_wbc_group_median"] = df.apply(
#     lambda row: (
#         overall_median_blood_wbc_by_y.loc[row["y"]]
#         if pd.isna(row["blood_wbc_group_median"])
#         else row["blood_wbc_group_median"]
#     ),
#     axis=1
# )

# # --------------------------------------------------
# #  Fill NA
# # --------------------------------------------------
# df["blood_wbc"] = df["blood_wbc"].fillna(
#     df["blood_wbc_group_median"]
# )

# # --------------------------------------------------
# #  Drop helper column
# # --------------------------------------------------
# df = df.drop(columns=["blood_wbc_group_median"])

# # --------------------------------------------------
# # Missing summary
# # --------------------------------------------------
# null_count = df["blood_wbc"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("blood_wbc Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("blood_wbc imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


blood_wbc imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## creatinine Missing Values handling

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [14]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# import pandas as pd
# import numpy as np

# # 0) Ensure numeric
# df["creatinine"] = pd.to_numeric(df["creatinine"], errors="coerce")

# # 1) Make age_group if not already present
# if "age_group" not in df.columns:
#     df["age_group"] = pd.cut(
#         df["anchor_age"],
#         bins=[0, 18, 40, 65, 120],
#         labels=["child", "adult", "middle_age", "elderly"]
#     )

# # 2) Sort so ffill/bfill within patient makes sense (class-aware)
# df = df.sort_values(["y", "subject_id", "admittime"])

# # 3) Patient-wise forward then backward fill within (y, subject_id)
# df["creatinine"] = df.groupby(["y", "subject_id"])["creatinine"].ffill()
# df["creatinine"] = df.groupby(["y", "subject_id"])["creatinine"].bfill()

# # 4) Compute group median by (y, age_group, gender)
# group_med = (
#     df.groupby(["y", "age_group", "gender"], observed=False)["creatinine"]
#       .median()
# )

# # 5) Join group median back on df
# df = df.join(
#     group_med.rename("creatinine_group_median"),
#     on=["y", "age_group", "gender"]
# )

# # 6) Fallback overall median within each y
# overall_med_by_y = df.groupby("y")["creatinine"].median()

# df["creatinine_group_median"] = df.apply(
#     lambda row: (
#         overall_med_by_y.loc[row["y"]]
#         if pd.isna(row["creatinine_group_median"])
#         else row["creatinine_group_median"]
#     ),
#     axis=1
# )

# # 7) Fill remaining missing creatinine
# df["creatinine"] = df["creatinine"].fillna(
#     df["creatinine_group_median"]
# )

# # 8) Drop helper column
# df = df.drop(columns=["creatinine_group_median"])

# # --------------------------------------------------
# # Missing summary
# # --------------------------------------------------
# null_count = df["creatinine"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("creatinine Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("creatinine imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


creatinine imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## Temperature Missing values Handling

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [16]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# # --------------------------------------------------
# # Sort by class, patient and time
# # --------------------------------------------------
# df = df.sort_values(["y", "subject_id", "admittime"])

# # --------------------------------------------------
# #  Forward fill within (y, subject_id)
# # --------------------------------------------------
# df["temperature"] = df.groupby(["y", "subject_id"])["temperature"].ffill()

# # --------------------------------------------------
# #  Backward fill within (y, subject_id)
# # --------------------------------------------------
# df["temperature"] = df.groupby(["y", "subject_id"])["temperature"].bfill()


# # --------------------------------------------------
# # Missing summary after temporal fill
# # --------------------------------------------------
# null_count = df["temperature"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("temperature Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")


# # --------------------------------------------------
# #  Group median by (y, age_group, gender)
# # --------------------------------------------------
# group_median = (
#     df.groupby(["y", "age_group", "gender"], observed=False)["temperature"]
#       .median()
# )

# # --------------------------------------------------
# # Join group median onto df
# # --------------------------------------------------
# df = df.join(
#     group_median.rename("temperature_group_median"),
#     on=["y", "age_group", "gender"]
# )

# # --------------------------------------------------
# # Fallback median within each y
# # --------------------------------------------------
# overall_median_by_y = df.groupby("y")["temperature"].median()

# df["temperature_group_median"] = df.apply(
#     lambda row: (
#         overall_median_by_y.loc[row["y"]]
#         if pd.isna(row["temperature_group_median"])
#         else row["temperature_group_median"]
#     ),
#     axis=1
# )

# # --------------------------------------------------
# #  Fill remaining missing temperature values
# # --------------------------------------------------
# df["temperature"] = df["temperature"].fillna(
#     df["temperature_group_median"]
# )

# # --------------------------------------------------
# # Drop helper column
# # --------------------------------------------------
# df = df.drop(columns=["temperature_group_median"])


# # --------------------------------------------------
# # Final missing summary
# # --------------------------------------------------
# null_count = df["temperature"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("temperature Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("temperature imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


temperature imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## heart_rate missing values

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [18]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# import pandas as pd
# import numpy as np

# # --------------------------------------------------
# # Safety: ensure admittime datetime
# # --------------------------------------------------
# df["admittime"] = pd.to_datetime(df["admittime"], errors="coerce")

# # --------------------------------------------------
# # 1️⃣ Sort by class, patient, and time
# # --------------------------------------------------
# df = df.sort_values(["y", "subject_id", "admittime"])

# # --------------------------------------------------
# # 2️⃣ Patient-wise forward then backward fill within (y, subject_id)
# # --------------------------------------------------
# df["heart_rate"] = df.groupby(["y", "subject_id"])["heart_rate"].ffill()
# df["heart_rate"] = df.groupby(["y", "subject_id"])["heart_rate"].bfill()

# # --------------------------------------------------
# # Missing summary after temporal fill
# # --------------------------------------------------
# null_count = df["heart_rate"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("heart_rate Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

# # --------------------------------------------------
# # 3️⃣ Ensure age_group exists
# # --------------------------------------------------
# if "age_group" not in df.columns:
#     df["age_group"] = pd.cut(
#         df["anchor_age"],
#         bins=[0, 18, 40, 65, 120],
#         labels=["child", "adult", "middle_age", "elderly"]
#     )

# # --------------------------------------------------
# # 4️⃣ Group median by (y, age_group, gender)
# # --------------------------------------------------
# group_median = (
#     df.groupby(["y", "age_group", "gender"], observed=False)["heart_rate"]
#       .median()
# )

# # --------------------------------------------------
# # 5️⃣ Join group median onto df
# # --------------------------------------------------
# df = df.join(
#     group_median.rename("hr_group_median"),
#     on=["y", "age_group", "gender"]
# )

# # --------------------------------------------------
# # 6️⃣ Fallback median within each y
# # --------------------------------------------------
# overall_median_by_y = df.groupby("y")["heart_rate"].median()

# df["hr_group_median"] = df.apply(
#     lambda row: (
#         overall_median_by_y.loc[row["y"]]
#         if pd.isna(row["hr_group_median"])
#         else row["hr_group_median"]
#     ),
#     axis=1
# )

# # --------------------------------------------------
# # 7️⃣ Fill remaining missing heart_rate values
# # --------------------------------------------------
# df["heart_rate"] = df["heart_rate"].fillna(df["hr_group_median"])

# # --------------------------------------------------
# # 8️⃣ Drop helper column
# # --------------------------------------------------
# df = df.drop(columns=["hr_group_median"])

# # --------------------------------------------------
# # Final missing summary
# # --------------------------------------------------
# null_count = df["heart_rate"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("heart_rate Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("heart_rate imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


heart_rate imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## resp_rate missing values handling

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [20]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# import numpy as np
# import pandas as pd

# # --------------------------------------------------
# # Safety: ensure admittime datetime
# # --------------------------------------------------
# df["admittime"] = pd.to_datetime(df["admittime"], errors="coerce")

# # --------------------------------------------------
# # Sort by class, patient, and time
# # --------------------------------------------------
# df = df.sort_values(["y", "subject_id", "admittime"])

# # --------------------------------------------------
# # Patient-wise forward then backward fill within (y, subject_id)
# # --------------------------------------------------
# df["resp_rate"] = df.groupby(["y", "subject_id"])["resp_rate"].ffill()
# df["resp_rate"] = df.groupby(["y", "subject_id"])["resp_rate"].bfill()

# # --------------------------------------------------
# # Missing summary after temporal fill
# # --------------------------------------------------
# null_count = df["resp_rate"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("resp_rate Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")


# # --------------------------------------------------
# # Ensure age_group exists
# # --------------------------------------------------
# if "age_group" not in df.columns:
#     df["age_group"] = pd.cut(
#         df["anchor_age"],
#         bins=[0, 18, 40, 65, 120],
#         labels=["child", "adult", "middle_age", "elderly"]
#     )

# # --------------------------------------------------
# # Group median by (y, age_group, gender)
# # --------------------------------------------------
# group_median = (
#     df.groupby(["y", "age_group", "gender"], observed=False)["resp_rate"]
#       .median()
# )

# # --------------------------------------------------
# # Join group median onto df
# # --------------------------------------------------
# df = df.join(
#     group_median.rename("rr_group_median"),
#     on=["y", "age_group", "gender"]
# )

# # --------------------------------------------------
# #  Fallback median within each y
# # --------------------------------------------------
# overall_median_by_y = df.groupby("y")["resp_rate"].median()

# df["rr_group_median"] = df.apply(
#     lambda row: (
#         overall_median_by_y.loc[row["y"]]
#         if pd.isna(row["rr_group_median"])
#         else row["rr_group_median"]
#     ),
#     axis=1
# )

# # --------------------------------------------------
# #  Fill remaining NaNs
# # --------------------------------------------------
# df["resp_rate"] = df["resp_rate"].fillna(df["rr_group_median"])

# # --------------------------------------------------
# #  Drop helper column
# # --------------------------------------------------
# df = df.drop(columns=["rr_group_median"])


# # --------------------------------------------------
# # Final missing summary
# # --------------------------------------------------
# null_count = df["resp_rate"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("resp_rate Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("resp_rate imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


resp_rate imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## o2sat Missing Values Handling

**NOTE: This section is now handled by advanced K-NN imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation.

In [22]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced K-NN imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# ============================================================================

# import pandas as pd
# import numpy as np

# # --------------------------------------------------
# # ensure admittime is datetime
# # --------------------------------------------------
# df["admittime"] = pd.to_datetime(df["admittime"], errors="coerce")

# # --------------------------------------------------
# # patient-wise time-ordered ffill then bfill
# #     (class-aware)
# # --------------------------------------------------
# df = df.sort_values(["y", "subject_id", "admittime"])
# df["o2sat"] = df.groupby(["y", "subject_id"])["o2sat"].ffill()
# df["o2sat"] = df.groupby(["y", "subject_id"])["o2sat"].bfill()

# # --------------------------------------------------
# # Missing summary after temporal fill
# # --------------------------------------------------
# null_count = df["o2sat"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("o2sat Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")


# # --------------------------------------------------
# #  ensure age_group exists
# # --------------------------------------------------
# if "age_group" not in df.columns:
#     df["age_group"] = pd.cut(
#         df["anchor_age"],
#         bins=[0, 18, 40, 65, 120],
#         labels=["child", "adult", "middle_age", "elderly"]
#     )

# # --------------------------------------------------
# #  group median by (y, age_group, icu_admission)
# # --------------------------------------------------
# group_median = (
#     df.groupby(
#         ["y", "age_group", "icu_admission"],
#         observed=False
#     )["o2sat"]
#     .median()
# )

# # --------------------------------------------------
# #  join group median back onto df
# # --------------------------------------------------
# df = df.drop(columns=["o2_group_median"], errors="ignore")
# df = df.join(
#     group_median.rename("o2_group_median"),
#     on=["y", "age_group", "icu_admission"]
# )

# # --------------------------------------------------
# #  fallback median within each y
# # --------------------------------------------------
# overall_median_by_y = df.groupby("y")["o2sat"].median()

# df["o2_group_median"] = df.apply(
#     lambda row: (
#         overall_median_by_y.loc[row["y"]]
#         if pd.isna(row["o2_group_median"])
#         else row["o2_group_median"]
#     ),
#     axis=1
# )

# # --------------------------------------------------
# #  fill remaining missing values
# # --------------------------------------------------
# df["o2sat_filled"] = df["o2sat"].fillna(df["o2_group_median"])

# df["o2sat_filled"] = df["o2sat_filled"].fillna(
#     df["y"].map(overall_median_by_y)
# )
# # --------------------------------------------------
# #  final numeric coercion
# # --------------------------------------------------
# df["o2sat_final"] = pd.to_numeric(df["o2sat_filled"], errors="coerce")

# # --------------------------------------------------
# #  overwrite original column
# # --------------------------------------------------
# df["o2sat"] = df["o2sat_final"]

# # --------------------------------------------------
# #  cleanup helper columns
# # --------------------------------------------------
# df = df.drop(
#     columns=["o2_group_median", "o2sat_filled", "o2sat_final"],
#     errors="ignore"
# )

# # --------------------------------------------------
# # Final missing summary
# # --------------------------------------------------
# null_count = df["o2sat"].isna().sum()
# total = len(df)
# percent_null = round((null_count / total) * 100, 2)

# print("o2sat Null Count:", null_count)
# print("Total Rows:", total)
# print("Percentage Null:", percent_null, "%")

print("o2sat imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


o2sat imputation: Using advanced K-NN imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


## BP_systolic Missing Values Handling

**NOTE: This section is now handled by advanced MICE imputation in `3b_b2s_advanced_imputation_knn_mice.ipynb`**

The code below is commented out. Use the advanced imputation notebook for better relationship-aware imputation (models correlation between BP_systolic and BP_diastolic).

In [24]:
# ============================================================================
# COMMENTED OUT: Now handled by advanced MICE imputation
# See: scripts/bronze_to_silver/3b_b2s_advanced_imputation_knn_mice.ipynb
# MICE models the correlation between BP_systolic and BP_diastolic
# ============================================================================

# # --------------------------------------------------
# # Sort by class, patient, and time
# # --------------------------------------------------
# df = df.sort_values(["y", "subject_id", "admittime"])

# # --------------------------------------------------
# # Patient-wise forward & backward fill within (y, subject_id)
# # --------------------------------------------------
# df["BP_systolic"] = (
#     df.groupby(["y", "subject_id"])["BP_systolic"]
#       .ffill()
#       .bfill()
# )

# df["BP_diastolic"] = (
#     df.groupby(["y", "subject_id"])["BP_diastolic"]
#       .ffill()
#       .bfill()
# )

# # --------------------------------------------------
# # Missing summary after temporal fill
# # --------------------------------------------------
# for col in ["BP_systolic", "BP_diastolic"]:
#     null_count = df[col].isna().sum()
#     total = len(df)
#     percent_null = round((null_count / total) * 100, 2)
#     print(f"{col} Null Count:", null_count)
#     print("Total Rows:", total)
#     print("Percentage Null:", percent_null, "%")
#     print()


# # --------------------------------------------------
# # Ensure age_group exists
# # --------------------------------------------------
# df["age_group"] = pd.cut(
#     df["anchor_age"],
#     bins=[0, 18, 40, 65, 120],
#     labels=["child", "adult", "middle_age", "elderly"]
# )

# # --------------------------------------------------
# # Compute group medians by (y, age_group, icu_admission)
# # --------------------------------------------------
# bp_sys_group = (
#     df.groupby(
#         ["y", "age_group", "icu_admission"],
#         observed=False
#     )["BP_systolic"]
#     .median()
# )

# bp_dias_group = (
#     df.groupby(
#         ["y", "age_group", "icu_admission"],
#         observed=False
#     )["BP_diastolic"]
#     .median()
# )

# # --------------------------------------------------
# # Drop old helper columns if re-running
# # --------------------------------------------------
# df = df.drop(
#     columns=["bp_sys_group_median", "bp_dias_group_median"],
#     errors="ignore"
# )

# # --------------------------------------------------
# # Join group medians back
# # --------------------------------------------------
# df = df.join(
#     bp_sys_group.rename("bp_sys_group_median"),
#     on=["y", "age_group", "icu_admission"]
# )

# df = df.join(
#     bp_dias_group.rename("bp_dias_group_median"),
#     on=["y", "age_group", "icu_admission"]
# )

# # --------------------------------------------------
# # Fallback median within each y
# # --------------------------------------------------
# sys_overall_by_y  = df.groupby("y")["BP_systolic"].median()
# dias_overall_by_y = df.groupby("y")["BP_diastolic"].median()

# df["bp_sys_group_median"] = df.apply(
#     lambda r: sys_overall_by_y.loc[r["y"]]
#     if pd.isna(r["bp_sys_group_median"]) else r["bp_sys_group_median"],
#     axis=1
# )

# df["bp_dias_group_median"] = df.apply(
#     lambda r: dias_overall_by_y.loc[r["y"]]
#     if pd.isna(r["bp_dias_group_median"]) else r["bp_dias_group_median"],
#     axis=1
# )

# # --------------------------------------------------
# # Fill remaining missing values
# # --------------------------------------------------
# df["BP_systolic"]  = df["BP_systolic"].fillna(df["bp_sys_group_median"])
# df["BP_diastolic"] = df["BP_diastolic"].fillna(df["bp_dias_group_median"])

# # --------------------------------------------------
# # Cleanup helper columns
# # --------------------------------------------------
# df = df.drop(columns=["bp_sys_group_median", "bp_dias_group_median"])

# # --------------------------------------------------
# # Final missing summary
# # --------------------------------------------------
# for col in ["BP_systolic", "BP_diastolic"]:
#     null_count = df[col].isna().sum()
#     total = len(df)
#     percent_null = round((null_count / total) * 100, 2)
#     print(f"{col} Null Count:", null_count)
#     print("Total Rows:", total)
#     print("Percentage Null:", percent_null, "%")
#     print()

print("BP_systolic and BP_diastolic imputation: Using advanced MICE imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)")


BP_systolic and BP_diastolic imputation: Using advanced MICE imputation (see 3b_b2s_advanced_imputation_knn_mice.ipynb)


In [25]:
missing_only = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percentage": (df.isna().mean() * 100).round(2)
})

# Keep only columns that actually have missing values
missing_only = missing_only[missing_only["missing_count"] > 0]

# Sort by highest missing percentage
missing_only = missing_only.sort_values("missing_percentage", ascending=False)

print(missing_only)

                        missing_count  missing_percentage
urine_bacteria                 157797               99.86
catheter_duration_days         156447               99.00
final_removal_date             156260               98.89
blood_crp                      147072               93.07
cfu_count                      145635               92.16
urine_output_ml                137052               86.73
final_insertion_date           130051               82.30
catheter_size                  105328               66.65
urinalysis_nitrite              89116               56.40
BP_systolic                     55651               35.22
BP_diastolic                    55647               35.22
temperature                     47963               30.35
o2sat                           47659               30.16
resp_rate                       47067               29.79
heart_rate                      47020               29.76
discharge_location              15245                9.65
creatinine    

## discharge_location missing values handling 

In [27]:
# Compute mode per class y
mode_by_y = (
    df.groupby("y")["discharge_location"]
      .agg(lambda x: x.mode(dropna=True).iloc[0] if not x.mode(dropna=True).empty else np.nan)
)

# Fill missing values using class-specific mode
df["discharge_location"] = df["discharge_location"].fillna(
    df["y"].map(mode_by_y)
)


In [28]:
df["discharge_location"].isna().sum()


0

## urinalysis_nitrite handling missing values

In [30]:
import numpy as np

#  Flag: was nitrite test performed?
df["nitrite_tested"] = df["urinalysis_nitrite"].notna().astype(int)

# Flag: nitrite positive (only 1 if tested and positive)
df["nitrite_positive"] = (df["urinalysis_nitrite"] == 1).astype(int)


In [31]:
df = df.drop(columns=["urinalysis_nitrite"])

In [32]:
df[["nitrite_tested", "nitrite_positive"]].isna().sum()


nitrite_tested      0
nitrite_positive    0
dtype: int64

## Drop insertion and removal columns

In [34]:
df = df.drop(
    columns=["final_removal_date", "final_insertion_date","urine_bacteria"],
    errors="ignore"
)

In [35]:
missing_only = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percentage": (df.isna().mean() * 100).round(2)
})

# Keep only columns that actually have missing values
missing_only = missing_only[missing_only["missing_count"] > 0]

# Sort by highest missing percentage
missing_only = missing_only.sort_values("missing_percentage", ascending=False)

print(missing_only)

                        missing_count  missing_percentage
catheter_duration_days         156447               99.00
blood_crp                      147072               93.07
cfu_count                      145635               92.16
urine_output_ml                137052               86.73
catheter_size                  105328               66.65
BP_systolic                     55651               35.22
BP_diastolic                    55647               35.22
temperature                     47963               30.35
o2sat                           47659               30.16
resp_rate                       47067               29.79
heart_rate                      47020               29.76
creatinine                      10339                6.54
blood_wbc                        9953                6.30


## create blood_crp_measured and cfu_count_measured

In [37]:
# Flag: was CRP measured?
df["blood_crp_measured"] = df["blood_crp"].notna().astype(int)

# Flag: was urine culture (CFU) measured?
df["cfu_count_measured"] = df["cfu_count"].notna().astype(int)
print(df["blood_crp_measured"].value_counts())
print(df["cfu_count_measured"].value_counts())
df = df.drop(columns=["blood_crp", "cfu_count"], errors="ignore")


blood_crp_measured
0    147072
1     10948
Name: count, dtype: int64
cfu_count_measured
0    145635
1     12385
Name: count, dtype: int64


## urine_output_ml handle missing values 

In [39]:
# Create measured flag
df["urine_output_measured"] = df["urine_output_ml"].notna().astype(int)

# Drop the raw column
df = df.drop(columns=["urine_output_ml"], errors="ignore")


In [40]:
cols = [
    "catheter_duration_days",
    "catheter_size"
]

summary_by_y = (
    df
    .groupby("y")[cols]
    .apply(lambda x: pd.DataFrame({
        "missing_count": x.isna().sum(),
        "non_missing_count": x.notna().sum(),
        "missing_percentage": (x.isna().mean() * 100).round(2)
    }))
)

print(summary_by_y)

                          missing_count  non_missing_count  missing_percentage
y                                                                             
0 catheter_duration_days         154627               1558               99.00
  catheter_size                  104018              52167               66.60
1 catheter_duration_days           1820                 15               99.18
  catheter_size                    1310                525               71.39


## catheter_duration_days handle missing values 

In [42]:
df["catheter_duration_measured"] = df["catheter_duration_days"].notna().astype(int)
df = df.drop(columns=["catheter_duration_days"], errors="ignore")

In [43]:
df["catheter_duration_measured"].value_counts()

catheter_duration_measured
0    156447
1      1573
Name: count, dtype: int64

## catheter_size handle missing values

In [45]:
df["catheter_size"] = df["catheter_size"].fillna("Unknown")

In [46]:
missing_only = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percentage": (df.isna().mean() * 100).round(2)
})

# Keep only columns that actually have missing values
missing_only = missing_only[missing_only["missing_count"] > 0]

# Sort by highest missing percentage
missing_only = missing_only.sort_values("missing_percentage", ascending=False)

print(missing_only)


              missing_count  missing_percentage
BP_systolic           55651               35.22
BP_diastolic          55647               35.22
temperature           47963               30.35
o2sat                 47659               30.16
resp_rate             47067               29.79
heart_rate            47020               29.76
creatinine            10339                6.54
blood_wbc              9953                6.30


In [47]:
df["catheter_size_known"] = (df["catheter_size"] != "Unknown").astype(int)

In [48]:
import re
import numpy as np

def extract_french_size(x):
    if pd.isna(x) or x == "Unknown":
        return np.nan
    m = re.search(r"(\d+)", str(x))
    return float(m.group(1)) if m else np.nan

df["catheter_size_fr"] = df["catheter_size"].apply(extract_french_size)


In [49]:
df["catheter_size_fr"] = df["catheter_size_fr"].clip(lower=6, upper=24)

In [50]:
df["catheter_size_measured"] = df["catheter_size_fr"].notna().astype(int)

In [51]:
df = df.drop(columns=["catheter_size"])

In [52]:
save_path = PROJECT_ROOT / "data" / "silver" / "bronze_missing_values_handled.csv"

df.to_csv(save_path, index=False)

print("Saved to:", output_path)

Saved to: C:\Users\Coditas\Desktop\Projects\Cauti\data\silver\missing_summary.csv
